In [2]:
import glob
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# Traditional Features Construction

In [2]:
def get_image():
    path = "../DataSet/Training/Normal"
    folders = os.listdir(path)
    no_of_images = []
    for folder in folders:
        files = [f for f in os.listdir(f"{path}/{folder}") if os.path.isfile(os.path.join(f"../Resized/{folder}", f))]
        no_of_images.append(len(files))
        
    collection = {}
    for folder, count in zip(folders, no_of_images):
        all_files = os.listdir(f"{path}/{folder}")
        collection[folder] = all_files
        
    return collection

In [3]:
image_collections = get_image()

In [4]:
def feature_extraction(path, new_path, folder, image):
    path = path + "/" + folder + "/" + image
    loaded_image = cv.imread(path)
    gray = cv.cvtColor(loaded_image, cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)
    img = cv.drawKeypoints(gray, kp, loaded_image)
    new_path = new_path + "/" + folder
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    new_path = new_path + "/" + image
    cv.imwrite(new_path, img)

In [5]:
def get_features():
    new_path = "../DataSet/Training/Feature Extracted"
    path = "../DataSet/Training/Normal"
    if not os.path.exists(path):
        os.mkdir(path)
    for folder, images in image_collections.items():
        for image in images:
            feature_extraction(path, new_path, folder, image)

In [6]:
get_features()

In [9]:
training_images = glob.glob("../DataSet/Training/Feature Extracted/*/*")
testing_images = glob.glob("../DataSet/Testing/*/*")

In [10]:
def get_pixels(images):
    combine = []
    for img in images:
        loaded_image = cv.imread(img)
        combine.append(loaded_image)
    
    return combine

In [13]:
training_images = get_pixels(training_images)
testing_images = get_pixels(testing_images)

In [7]:
def k_means_clustering(X):
    kmeans = KMeans(n_clusters=100, random_state=0, init="random", n_init="auto").fit(X)
    
    return kmeans

In [ ]:
def dimension_reduction(features):
    pca = PCA(n_components=2)
    transformed_data = pca.fit_transform(features)
    
    for i in range(4):
        c = np.flatnonzero(class_labels == i)
        plt.scatter(transformed_data[c, 0], transformed_data[c, 1])
    plt.show()
    
    return transformed_data

# Traditional Machine Learnign model - Support Vecotr Machine (SVM)

In [ ]:
def train_svm(kernel_type="linear"):
    parameters = [0.001, 0.1, 1.0, 10, 100]
    
    for c in parameters:
        svc = SVC(kernel=kernel_type, C=c, random_state=42)

In [ ]:
def plot_svm_performance():
    plt.plot()

In [ ]:
def train_svm_for_different_kernels():
    kernel_type = "rbf", "poly", "sigmoid"
    for kernel in kernel_type:
        train_svm(kernel_type)

# Deep Learning - Training a Simple Convolution Neural Network Mode

In [18]:
def images_classifier():
    num_nodes = 10
    input_shape = (100, 100, 1)
    
    model = keras.Sequential(
    [
        keras.Input(shape=input_shape), 
        layers.Conv2D(8, kernel_size=(3, 3), activation="relu"), 
        layers.Conv2D(8, kernel_size=(3, 3), activation="relu"), 
        layers.MaxPooling2D(pool_size=(2, 2)), 
        layers.Flatten(), 
        layers.Dense(16, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(num_nodes, activation="softmax")
    ])
    
    model.summary()
    
    return model

In [19]:
model = images_classifier()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 98, 98, 8)         80        
                                                                 
 conv2d_3 (Conv2D)           (None, 96, 96, 8)         584       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 8)         0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 18432)             0         
                                                                 
 dense_3 (Dense)             (None, 16)                294928    
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                      

In [15]:
def train_model():
    batch_size = 128
    num_epochs = 30
    X_train, y_train, X_test, y_test = train_test_split(training_images, testing_images, test_size=0.2)
    
    X_train = np.reshape(X_train, (X_train.shape[0], 100, 100, 1))
    X_test = np.reshape(X_test, (X_test.shape[0], 100, 100, 1))
    
    y_train = to_categorical(y_train, num_classes=10)
    y_test = to_categorical(y_test, num_classes=10)
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.2)
    
    return model, history, X_test, y_test

In [16]:
def model_evaluation():
    model, history, X_test, y_test = train_model()
    score = model.evaluate(X_test, y_test, verbose=0)
    print("\n\nTest loss:", score[0])
    print("Test accuracy:", score[1])
    
    return history

In [17]:
model = model_evaluation()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 8)         80        
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 8)         584       
                                                                 
 max_pooling2d (MaxPooling2  (None, 48, 48, 8)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 16)                294928    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                      

NameError: name 'standardized_dataset' is not defined

# Transfer Learning via Feature Extraction